In [2]:
import pandas as pd
from IPython.display import display, Markdown, Latex

import textstat
import tabulate

In [3]:
KEYS_TO_EXTRACT = ('participantID', 'timestamp', 'scene', 
    'game.setup', 'game.gameplay', 'game.scoring', 'game.difficulty', 'game.firstTimeScore', 
    'gameScore.score', 'gameScore.thoughts', 
    'debrief.strategy', 'debrief.difficulties', 'debrief.questions', 'debrief.external_aids')

SHORT_SCENE_NAMES = {
    'FloorPlan326_physics_semi_sparse_few_new_objects': 'few_objects',
    'FloorPlan326_physics_semi_sparse_new_objects': 'medium_objects',
    'FloorPlan326_physics_semi_sparse_many_new_objects': 'many_objects',
}

GAME_TEMPLATE = """(define (game {participantID}) (:domain {room}-objects-room-v1)  ; {index}
(:setup (and 

))
(:constraints (and 

))
(:scoring maximize

))"""

def recursive_extract_value(d, key):
    if '.' in key:
        split_key = key.split('.')

    else:
        split_key = [key]

    value = d
    for key_part in split_key:
        if key_part == 'game':
            key_part = 'editedGame' if 'editedGame' in value else 'initialGame'
                
        value = value[key_part] if key_part in value else None
        if value == None:
            return value

    return value

def participant_dict_to_row(doc, keys=KEYS_TO_EXTRACT):
    d = doc.to_dict()
    return [doc.id] + [recursive_extract_value(d, key) for key in keys]


def print_participant(df, index, game_fields=('game_setup', 'game_gameplay', 'game_scoring', 'game_difficulty', 'game_firstTimeScore')):
    p = df.loc[index]
    display(Markdown(f'## {p.participantID} ({p.id}) ({p.scene})'))
    for game_field in game_fields:
        display(Markdown(f'### **{game_field.replace("game_", "")}:** {p[game_field]}'))

    room = p.scene.split('_')[0] if p.scene is not None else ''
    print(GAME_TEMPLATE.format(participantID=p.participantID, room=room, index=index))

In [4]:
LOAD_DATA_FROM_FIRESTORE = False
WRITE_DATA_TO_CSV = False
PARTICIPANTS_CSV_PATH = '../data/interactive_beta.csv'
COLLECTION_NAME = 'participants-v2'


if LOAD_DATA_FROM_FIRESTORE:
    import firebase_admin
    from firebase_admin import credentials
    from firebase_admin import firestore

    # Use a service account
    cred = credentials.Certificate('./game-generation-6db9c-aef76f1917f8.json')
    firebase_admin.initialize_app(cred)

    db = firestore.client()

    participants_with_replays = db.collection(COLLECTION_NAME).order_by('timestamp', direction=firestore.Query.DESCENDING).order_by('replays', direction=firestore.Query.DESCENDING).stream()
    participant_rows = [participant_dict_to_row(doc) for doc in participants_with_replays]
    participant_df = pd.DataFrame(participant_rows, columns=['id'] + [key.replace('.', '_') for key in KEYS_TO_EXTRACT])
    participant_df.scene = [SHORT_SCENE_NAMES[s] if s in SHORT_SCENE_NAMES else None for s in participant_df.scene]
    participant_df = participant_df[participant_df.scene.notna() & participant_df.game_setup.notna() & participant_df.game_gameplay.notna() & participant_df.game_scoring.notna()]
    participant_df = participant_df[participant_df.participantID.str.len() > 10]
    participant_df = participant_df[participant_df.participantID != '5f63a8f17e0e2f0c5aebfc0b']
    participant_df = participant_df.reset_index()

    if WRITE_DATA_TO_CSV:
        participant_df.to_csv(PARTICIPANTS_CSV_PATH)

else:
    participant_df = pd.read_csv(PARTICIPANTS_CSV_PATH)



In [5]:
print(participant_df.shape)
participant_df.head()

(21, 17)


,Unnamed: 0,index,id,participantID,timestamp,scene,game_setup,game_gameplay,game_scoring,game_difficulty,game_firstTimeScore,gameScore_score,gameScore_thoughts,debrief_strategy,debrief_difficulties,debrief_questions,debrief_external_aids
0,0,0,qSbsQv2jOptuDovotsIE,5e2df2855e01ef3e5d01ab58,2021-11-09 15:42:14.716000+00:00,medium_objects,No setup needed.,"To play my game, you will use the building blo...","To score my game, you will get 1 point for eac...",2,5,6,It was a lot more difficult to stack the block...,Since you were very much limited by the object...,The frame rate could get low at times.,It would have been nice to be able to simply ...,No.
1,1,1,SdpyTxFIhhYu7wNwmHFa,60e93f64ec69ecdac3107555,2021-11-09 15:38:30.625000+00:00,medium_objects,"The player should move basketball, beachball a...",The player stands next to the door facing the ...,If the ball was succesfully thrown into the bi...,2,7 points,4 points,I think it is interesting but requires practic...,I considered the games I like playing myself -...,I did not have any technical difficulties!,I found this study to be one of the most inter...,I used pen and paper to write down the items I...
2,2,2,PJ2WCWCLedT8sDLrvn49,613e4bf960ca68f8de00e5e7,2021-11-08 21:55:19.879000+00:00,medium_objects,NaN,The pieces on the shelf between the two Window...,For each castle built in the correct order wil...,1,40,0,In real life this game would be very easy (tha...,I just looked at what was in the room and deci...,"Yes, the page broke and I had to redo it.",No,Print on a piece of paper the shape of the cas...
3,3,3,GLPtcvJUaHkUYK7iEPRq,613e4bf960ca68f8de00e5e7,2021-11-08 21:14:13.145000+00:00,medium_objects,NaN,The pieces on the shelf between the two window...,For each castle built in the correct order of ...,0,40,0,In real life this game would be very easy (tha...,NaN,NaN,NaN,NaN
4,4,4,yWwOhJSjWiuzNkkHzhft,616e4f7a16145200573161a6,2021-11-08 21:10:48.702000+00:00,few_objects,closed blinds\nthe curved ramp with the square...,roll the dodge ball onto the bin across the ra...,if a ball enters the box its a point,3,maybe 1 point in 5 tries,0,its too complicated i couldnt play well,tried to move the objects to where I needed to...,the objects were hard to move around \nI could...,none,took screenshot


In [6]:
print_participant(participant_df, 4)

## 616e4f7a16145200573161a6 (yWwOhJSjWiuzNkkHzhft) (few_objects)

### **setup:** closed blinds
the curved ramp with the square boxes on the side
the bin after the ramp

### **gameplay:** roll the dodge ball onto the bin across the ramp with the cube blocks around to direct the ball onto the ramp to the bin

### **scoring:** if a ball enters the box its a point

### **difficulty:** 3

### **firstTimeScore:** maybe 1 point in 5 tries

(define (game 616e4f7a16145200573161a6) (:domain few-objects-room-v1)  ; 4
(:setup (and 

))
(:constraints (and 

))
(:scoring maximize

))


# Statistics on plaintext versions

In [6]:
DEFAULT_STATS_FIELDS = ('game_setup', 'game_gameplay', 'game_scoring')

def run_textstat_func(textstat_func, df, fields=DEFAULT_STATS_FIELDS):
    if isinstance(fields, slice):
        subset = df.iloc[:, fields].copy()
    else:
        subset = df.loc[:, fields].copy()
        
    subset[subset.isna()] = ''
    s = subset.agg('\n'.join, axis=1)
    scores = s.apply(textstat_func)
    return scores.mean(), scores.std() / (len(scores) ** 0.5)

In [7]:
run_textstat_func(textstat.flesch_reading_ease, participant_df)

(75.63380952380952, 4.418337517351245)

In [8]:
few_objects_df = pd.read_csv('../data/few_objects.csv')
medium_objects_df = pd.read_csv('../data/medium_objects.csv')
many_objects_df = pd.read_csv('../data/many_objects.csv')

In [9]:
textstat.flesch_reading_ease.__name__

'flesch_reading_ease'

In [11]:
TEXTSTAT_FUNCS = (textstat.flesch_reading_ease, textstat.flesch_kincaid_grade, textstat.gunning_fog)
NAMES = ['survey - few objects', 'survey - medium objects', 'survey - many objects', 'interactive beta']

rows = []

for i, df in enumerate((few_objects_df, medium_objects_df, many_objects_df, participant_df)):
    scores = [run_textstat_func(func, df, fields=slice(1, 4) if i < 3 else DEFAULT_STATS_FIELDS) for func in TEXTSTAT_FUNCS]
    rows.append([NAMES[i]] + [f'$ {s[0]:.2f} \\pm {s[1]:.2f} $' for s in scores])

headers = ['name'] + [func.__name__ for func in TEXTSTAT_FUNCS]

display(Markdown(tabulate.tabulate(rows, headers=headers, tablefmt='github')))

# few_objects_df.iloc[:, slice(1, 4)].agg(lambda x: [type(z) for z in x], axis=1)

| name                    | flesch_reading_ease   | flesch_kincaid_grade   | gunning_fog        |
|-------------------------|-----------------------|------------------------|--------------------|
| survey - few objects    | $ 61.48 \pm 7.66 $    | $ 14.73 \pm 2.96 $     | $ 17.11 \pm 3.06 $ |
| survey - medium objects | $ 61.76 \pm 5.55 $    | $ 13.73 \pm 2.08 $     | $ 16.03 \pm 2.12 $ |
| survey - many objects   | $ 56.42 \pm 5.95 $    | $ 15.56 \pm 2.23 $     | $ 17.65 \pm 2.30 $ |
| interactive beta        | $ 75.63 \pm 4.42 $    | $ 9.39 \pm 1.59 $      | $ 11.83 \pm 1.57 $ |